RoPE 通过复数数乘的方式，让 token 在高维空间中进行旋转，从而编码相对位置信息。

对于输入向量 $x=\left(x_{1}, x_{2}, \ldots, x_{d}\right)$（其中 d 是维度），RoPE 将其拆分为偶数索引和奇数索引
$$\left(x_{1}, x_{2}\right),\left(x_{3}, x_{4}\right), \ldots,\left(x_{d-1}, x_{d}\right)$$

然后，对这些二维向量对进行旋转

$$\left(x^{\prime}, y^{\prime}\right)=(x \cos \theta-y \sin \theta, x \sin \theta+y \cos \theta)$$

中 $\theta$ 由位置 p 和固定基数 10000 计算得到
$$\theta_{i}=\frac{p}{10000^{2 i / d}}$$

适用于 Transformer，通常用于 query 和 key，相乘后会得到相对位置信息。增强注意力机制对相对位置信息的建模。

In [41]:
import torch

B, T, C = 2, 20, 128
x = torch.randn(B, T, C)
Wq, Wk = torch.nn.Linear(C, C), torch.nn.Linear(C, C)
Q, K = Wq(x), Wk(x)

theta = 10000 ** (-torch.arange(0, C, 2) / C)
theta = theta.expand(size=(20, 64))
theta.shape

torch.Size([20, 64])

In [42]:
pos = torch.arange(0, T).unsqueeze(1)
pos.shape

torch.Size([20, 1])

In [43]:
angles = (pos * theta)
cos = torch.cos(angles)
sin = torch.sin(angles)
cos.shape

torch.Size([20, 64])

In [51]:
x1 = Q[:, :, 0::2]
x2 = Q[:, :, 1::2]
rotated_x1 = x1 * cos - x2 * sin
rotated_x2 = x1 * sin + x2 * cos
Q_rope = torch.stack([rotated_x1, rotated_x2], dim=-1).flatten(-2)
Q_rope.shape

torch.Size([2, 20, 128])